In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [9]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [10]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.67810, saving model to best_m.h5
 - 2s - loss: 0.7010 - acc: 0.5483 - val_loss: 0.6781 - val_acc: 0.5212
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.6818 - acc: 0.5284 - val_loss: 0.6848 - val_acc: 0.5087
Epoch 3/50
Epoch 00003: val_loss improved from 0.67810 to 0.67769, saving model to best_m.h5
 - 1s - loss: 0.6687 - acc: 0.5633 - val_loss: 0.6777 - val_acc: 0.5087
Epoch 4/50
Epoch 00004: val_loss improved from 0.67769 to 0.54559, saving model to best_m.h5
 - 1s - loss: 0.6018 - acc: 0.6525 - val_loss: 0.5456 - val_acc: 0.7307
Epoch 5/50
Epoch 00005: val_loss improved from 0.54559 to 0.53688, saving model to best_m.h5
 - 1s - loss: 0.5796 - acc: 0.6761 - val_loss: 0.5369 - val_acc: 0.8304
Epoch 6/50
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.5436 - acc: 0.7191 - val_loss: 0.6412 - val_acc: 0.6608
Epoch 7/50
Epoch 00007: val_loss improved from 0.53688 to 0.42152, saving model to best_m.h5
 - 1s - l

Epoch 10/50
Epoch 00010: val_loss improved from 0.45482 to 0.41284, saving model to best_m.h5
 - 2s - loss: 0.4390 - acc: 0.8024 - val_loss: 0.4128 - val_acc: 0.7980
Epoch 11/50
Epoch 00011: val_loss improved from 0.41284 to 0.39363, saving model to best_m.h5
 - 1s - loss: 0.3815 - acc: 0.8256 - val_loss: 0.3936 - val_acc: 0.7980
Epoch 12/50
Epoch 00012: val_loss improved from 0.39363 to 0.38166, saving model to best_m.h5
 - 2s - loss: 0.3905 - acc: 0.8158 - val_loss: 0.3817 - val_acc: 0.8030
Epoch 13/50
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.3712 - acc: 0.8316 - val_loss: 0.3965 - val_acc: 0.7930
Epoch 14/50
Epoch 00014: val_loss improved from 0.38166 to 0.34570, saving model to best_m.h5
 - 1s - loss: 0.3454 - acc: 0.8500 - val_loss: 0.3457 - val_acc: 0.8279
Epoch 15/50
Epoch 00015: val_loss improved from 0.34570 to 0.33524, saving model to best_m.h5
 - 1s - loss: 0.3299 - acc: 0.8564 - val_loss: 0.3352 - val_acc: 0.8454
Epoch 16/50
Epoch 00016: val_loss did not improv

Epoch 19/50
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.3596 - acc: 0.8306 - val_loss: 0.3649 - val_acc: 0.8254
Epoch 20/50
Epoch 00020: val_loss improved from 0.28624 to 0.28532, saving model to best_m.h5
 - 2s - loss: 0.3463 - acc: 0.8541 - val_loss: 0.2853 - val_acc: 0.8753
Epoch 21/50
Epoch 00021: val_loss improved from 0.28532 to 0.27865, saving model to best_m.h5
 - 1s - loss: 0.3386 - acc: 0.8550 - val_loss: 0.2787 - val_acc: 0.8728
Epoch 22/50
Epoch 00022: val_loss improved from 0.27865 to 0.27489, saving model to best_m.h5
 - 1s - loss: 0.3670 - acc: 0.8481 - val_loss: 0.2749 - val_acc: 0.8778
Epoch 23/50
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.3366 - acc: 0.8500 - val_loss: 0.3637 - val_acc: 0.8204
Epoch 24/50
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.3574 - acc: 0.8426 - val_loss: 0.2815 - val_acc: 0.8778
Epoch 25/50
Epoch 00025: val_loss improved from 0.27489 to 0.27348, saving model to best_m.h5
 - 2s - loss: 0.3314 - acc: 0.8575 - val_l

Epoch 30/50
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.2348 - acc: 0.8998 - val_loss: 0.3418 - val_acc: 0.8554
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.2238 - acc: 0.9075 - val_loss: 0.3336 - val_acc: 0.8603
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.2281 - acc: 0.9033 - val_loss: 0.3456 - val_acc: 0.8653
Epoch 33/50
Epoch 00033: val_loss did not improve
 - 1s - loss: 0.2162 - acc: 0.9064 - val_loss: 0.3410 - val_acc: 0.8628
Epoch 34/50
Epoch 00034: val_loss did not improve
 - 1s - loss: 0.2319 - acc: 0.9000 - val_loss: 0.4053 - val_acc: 0.8354
Epoch 35/50
Epoch 00035: val_loss did not improve
 - 1s - loss: 0.1977 - acc: 0.9200 - val_loss: 0.3573 - val_acc: 0.8603
Epoch 36/50
Epoch 00036: val_loss did not improve
 - 1s - loss: 0.2230 - acc: 0.9000 - val_loss: 0.3190 - val_acc: 0.8703
Epoch 37/50
Epoch 00037: val_loss did not improve
 - 1s - loss: 0.2032 - acc: 0.9142 - val_loss: 0.3730 - val_acc: 0.8504
Epoch 38/50
Epoch 00038:

In [11]:
import pickle
with open('../features/cnn_1_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_aug1_sub.csv', index=False)

0.250823663322
         id  is_iceberg
0  5941774d    0.115767
1  4023181e    0.894766
2  b20200e4    0.014314
3  e7f018bb    0.977465
4  4371c8c3    0.458051
